In [1]:
import subprocess
import random
import os
import time

In [2]:
# Configurations
num_nodes = 23
min_transaction_amount = 1000  # Minimum transaction amount in satoshis
max_transaction_amount = 100000  # Maximum transaction amount in satoshis
transaction_interval = 5  # Time interval between transactions in seconds

# Additional configuration for block generation
block_generation_interval = 30  # Generate a block after every 30 transactions
transactions_since_last_block = 0


In [3]:
# Generate node information
nodes = [{'id': i, 'port': 10008 + i, 'macaroon_path': f'd:\\lnd\\node{i}\\data\\chain\\bitcoin\\regtest\\admin.macaroon', 'tls_cert_path': f'd:\\lnd\\node{i}\\tls.cert'} for i in range(2, num_nodes + 1)]

In [4]:
# Function to send a payment from one node to another
def send_payment(source_node, dest_node, amount):
    # Get the invoice from the destination node
    command = f'lncli --network regtest --macaroonpath={dest_node["macaroon_path"]} --tlscertpath={dest_node["tls_cert_path"]} --rpcserver=localhost:{dest_node["port"]} addinvoice {amount}'
    #print ("Command: ", command)
    result = subprocess.run(command, capture_output=True, text=True, shell=True)
    #print("Invoice command output:", result.stdout)
    
    if result.returncode != 0:
        print(f"Failed to generate invoice: {result.stderr}")
        return  # Skip sending the payment if invoice generation failed
    
    #print ("Invoice before split: ", result.stdout)
    
    invoice = result.stdout.split('"')[7]  # Extract the invoice
        
    # Send the payment from the source node
    command = f'lncli --network regtest --macaroonpath={source_node["macaroon_path"]} --tlscertpath={source_node["tls_cert_path"]} --rpcserver=localhost:{source_node["port"]} sendpayment --pay_req={invoice} --force'
    #print ("Running ", command)
    result = subprocess.run(command, shell=True)
    
    if result.returncode != 0:
        print(f"Failed to send payment: {result.stderr}")
        return


In [5]:
# Function to generate a block
def generate_block():
    command = 'bitcoin-cli -regtest -rpcport=18443 -rpcuser=myrpcusername -rpcpassword=myrpcpassword -generate 1'
    subprocess.run(command, shell=True)
    print("Generated a new block.")

In [6]:
generate_block()

# Infinite loop to send random payments
while True:
    source = random.choice(nodes[1:])  # Exclude node1 from being a source
    dest = random.choice(nodes)  # Any node can be a destination
    amount = random.randint(min_transaction_amount, max_transaction_amount)

    if source != dest:
        print(f"Sending {amount} sats from Node{source['id']} to Node{dest['id']}")
        try:
            send_payment(source, dest, amount)
        except Exception as e:
            print(f"Error sending payment: {e}")
            
        transactions_since_last_block += 1
        # Generate a block if the interval is reached
        if transactions_since_last_block >= block_generation_interval:
            generate_block()
            transactions_since_last_block = 0

    time.sleep(transaction_interval)

Generated a new block.
Sending 88875 sats from Node21 to Node11
Sending 31758 sats from Node4 to Node9
Sending 65531 sats from Node9 to Node2
Sending 98094 sats from Node8 to Node23
Sending 55237 sats from Node20 to Node8
Sending 49203 sats from Node13 to Node22
Sending 50958 sats from Node3 to Node4
Sending 84056 sats from Node13 to Node4
Sending 53831 sats from Node4 to Node18
Sending 62370 sats from Node20 to Node5
Sending 96404 sats from Node21 to Node11
Failed to send payment: None
Sending 87960 sats from Node4 to Node19
Sending 27823 sats from Node19 to Node8
Sending 63571 sats from Node3 to Node23
Sending 80897 sats from Node23 to Node4
Sending 22175 sats from Node11 to Node20
Sending 71729 sats from Node13 to Node6
Sending 27571 sats from Node16 to Node3
Sending 99433 sats from Node18 to Node5
Sending 70213 sats from Node20 to Node10
Sending 54901 sats from Node16 to Node9
Sending 52466 sats from Node14 to Node22
Sending 82096 sats from Node11 to Node12
Sending 7183 sats from N

KeyboardInterrupt: 